处理商品名称字段并语义化分词


In [194]:
product_name_1="加热鼠标垫大号办公室电脑桌面取暖桌垫宿舍学生做作业暖手垫写字8033cm 饼干熊款一个"
product_name_2="孕妇连衣裙夏款2025网红潮妈孕妇装夏季翻领T恤时尚洋气宽松裙子杏色"
product_name_3="特仑苏 有机纯牛奶 250ml10盒_箱"

In [195]:
import re

def clean_text(text):
    """
    清洗文本，去除特殊字符和多余空格
    """
    if not isinstance(text, str):
        return ""
    # 移除非中文字符、字母和数字
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9]', ' ', text)
    # 将多个空格合并为一个
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

In [196]:
import jieba

jieba.load_userdict("./jieba_dict.txt")
def chinese_tokenize(text: str) -> set:
    """中文精确分词，返回去重后的分词集合（适配Jaccard相似度计算）"""
    if not text or text.isspace():
        return set()
    return set(jieba.cut(text, cut_all=False))

In [197]:
tokenize_1 = chinese_tokenize(clean_text(product_name_1))
tokenize_2 = chinese_tokenize(clean_text(product_name_2))
tokenize_3 = chinese_tokenize(clean_text(product_name_3))


In [198]:
"|".join(tokenize_1)

' |大|垫|加热|取暖|8033cm|写字|暖手|号|电脑桌面|一个|做作业|学生|桌垫|饼干|办公室|鼠标垫|熊款|宿舍'

In [199]:
"|".join(tokenize_2)


'时尚洋气|恤|宽松裙子|夏季|2025|t|夏款|杏色|孕妇装|翻领|网红潮妈|孕妇连衣裙'

In [200]:
"|".join(tokenize_3)


' |250ml10|盒|特仑苏|箱|有机纯牛奶'

In [201]:
import re

def extract_full_spec_and_clean(text):
    """
    从商品名称中提取完整规格串，并返回去除规格后的文本。

    返回:
        spec, cleaned_text
    示例:
        输入:  "加热鼠标垫...8033cm 饼干熊款一个"
        输出:  ("8033cm", "加热鼠标垫... 饼干熊款一个")
    """
    if not isinstance(text, str):
        return ""

    # 单位字典（小写匹配）
    units = [
        "盒","箱","件","个","袋","包","瓶","罐","卷","片","只",
        "ml","l","g","kg","mm","cm","m"
    ]
    # 包装单位（可扩展）
    pack_units = ["盒","箱","件","个","袋","包","瓶","罐","卷","片","只","条","支"]
    units_pattern = "|".join(re.escape(u) for u in units)
    pack_pattern = "|".join(re.escape(p) for p in pack_units)

    # ✅ 动态正则模板
    pattern = re.compile(
        rf'((?:\d+(?:\.\d+)?\s?(?:{units_pattern}|{pack_pattern}))'
        rf'(?:[*_×xX/\\-]?\s?\d*(?:\.\d+)?\s?(?:{units_pattern}|{pack_pattern}))*)',
        flags=re.IGNORECASE
    )

    matches = pattern.findall(text)
    if not matches:
        return ""

    # 取最长匹配作为规格串
    specs = sorted(matches, key=len, reverse=True)
    spec = specs[0].strip()

    return spec


In [202]:
extract_full_spec_and_clean(product_name_1)

'8033cm'

In [203]:
extract_full_spec_and_clean(product_name_2)


''

In [204]:
extract_full_spec_and_clean(product_name_3)

'250ml10盒_箱'

In [205]:
from typing import List, Set


# 提取品牌 如输入特仑苏 有机纯牛奶 250ml10盒_箱 返回特仑苏
def extract_brand_from_tokens(text_to_search: str, brand_dictionary: Set[str]) -> str:
    """
    从jieba分词后的词语列表中，根据品牌词典提取品牌名。

    该算法遵循“最长匹配”原则，优先匹配由更多词组成的品牌名。

    参数:
    - tokenized_words (List[str]): jieba.lcut() 分词后得到的词语列表。
    - brand_dictionary (Set[str]): 包含所有已知品牌名的集合，便于快速查找。

    返回:
    - str: 匹配到的品牌名。如果没有匹配到，则返回 "未知"。
    """
    if not text_to_search or text_to_search.isspace() or not brand_dictionary:
        return ""

    for brand_info in brand_dictionary:
        # 直接在拼接后的字符串中查找，这比复杂的列表滑动窗口更高效简洁
        if brand_info in text_to_search:
            return brand_info # 找到第一个（也是最长的）匹配项，立即返回
            
    return ""



In [206]:
BRAND_DICTIONARY = {
        "海氏海诺",
        "爱斐堡",
        "汤达人",
        "康师傅",
        "上好佳",
        "海天",
        "老干妈",
        "可口可乐",
        "特仑苏",
        "华为",
        "华为荣耀",
    "味全"
    }

In [207]:
extract_brand_from_tokens(clean_text(product_name_3), BRAND_DICTIONARY)
extract_brand_from_tokens(clean_text("哒哒海氏海诺滴眼液300ml"), BRAND_DICTIONARY)

'海氏海诺'

In [208]:
# 获取商品名称字段的品牌/规格/分词三元组
def get_product_info(product_name: str) -> tuple:
    spec = extract_full_spec_and_clean(product_name)
    brand = extract_brand_from_tokens(clean_text(product_name), BRAND_DICTIONARY)
    tokenize = chinese_tokenize(clean_text(product_name))
    return brand, spec, tokenize

In [209]:
get_product_info(product_name_3)

('特仑苏', '250ml10盒_箱', {' ', '250ml10', '有机纯牛奶', '特仑苏', '盒', '箱'})

In [210]:
def calculate_brand_similarity(brand1: str, brand2: str) -> float:
    """
    计算两个品牌字符串的相似度。
    - 完全相同: 1.0
    - 任意一个未知: 0.5
    - 完全不同或双方: 0.0
    """
    brand1 = brand1.strip().lower()
    brand2 = brand2.strip().lower()

    if not brand1 or not brand2:
        return 0.5  
    if brand1 == brand2:
        return 1.0
    
    return 0.0

In [211]:
def jaccard_similarity(set1: set, set2: set) -> float:
    """计算Jaccard相似度：交集大小 / 并集大小（取值0-1，越大越相似）"""
    if not set1 and not set2:
        return 1.0  # 两个空字符串视为完全相似
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0.0

In [212]:
calculate_brand_similarity("华为", "华为荣耀")

0.0

In [213]:
calculate_brand_similarity("", "")


0.5

In [214]:
calculate_brand_similarity("华为", "")


0.5

In [215]:
calculate_brand_similarity("华为", "华为")


1.0

In [221]:
from typing import Tuple
import pandas as pd

WEIGHTS = {
        'similarity_tokens': 0.7,  # 品牌权重占 60%
        'similarity_brand': 0.3,   # 商品名权重占 40%
    }
def find_top3_similar_combined(product_name: str, candidate_df: pd.DataFrame, candidate_tokens_list: List[set]) -> Tuple[str, str, str]:
    p_name = clean_text(product_name)
    p_tokenize = chinese_tokenize(p_name)
    similarity_results = []
    p_brand=extract_brand_from_tokens(p_name,BRAND_DICTIONARY)

    for idx in range(len(candidate_df)):
        candidate_id = candidate_df.iloc[idx]['商品ID']
        candidate_name = candidate_df.iloc[idx]['商品名称']
        candidate_tokens = candidate_tokens_list[idx]
        candidate_brand= extract_brand_from_tokens(candidate_name,BRAND_DICTIONARY)
        
        similarity_tokens = jaccard_similarity(p_tokenize, candidate_tokens)
        similarity_brand = calculate_brand_similarity(p_brand, candidate_brand)
        similarity=similarity_tokens * WEIGHTS['similarity_tokens'] + similarity_brand * WEIGHTS['similarity_brand']
        # 存储（负相似度：用于升序排序等价于相似度降序，商品ID-名称组合）
        similarity_results.append((-similarity, f"{candidate_id}-{candidate_name}--{similarity}--{similarity_tokens}--{similarity_brand}"))
    
        # 按相似度降序排序（相同相似度保留附件2原始顺序）
    similarity_results.sort()
    # 提取Top3组合，不足3个时用空字符串填充
    top3_combined = [result[1] for result in similarity_results[:3]]
    while len(top3_combined) < 3:
        top3_combined.append("")  # 补空确保始终返回3个元素

    return top3_combined[0], top3_combined[1], top3_combined[2]

In [217]:
def preprocess_candidate_tokens(candidate: pd.DataFrame) -> List[set]:
    return [chinese_tokenize(clean_text(name)) for name in candidate['商品名称']]

In [218]:
def load_excel(file_path: str) -> pd.DataFrame:
    """
    读取Excel文件，验证必要列并处理空值
    要求文件必须包含"商品ID"和"商品名称"列（可修改required_cols适配实际列名）
    """
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
        required_cols = ['商品ID', '商品名称']
        # 检查必要列是否存在
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"缺少必要列：{', '.join(missing_cols)}，需包含{required_cols}")
        # 去除商品ID或名称为空的无效行
        df = df.dropna(subset=required_cols).reset_index(drop=True)
        # 强制转为字符串类型，避免数字ID/名称拼接出错（如科学计数法、格式丢失）
        df['商品ID'] = df['商品ID'].astype(str).str.strip()
        df['商品名称'] = df['商品名称'].astype(str).str.strip()
        return df
    except Exception as e:
        print(f"读取文件{file_path}失败：{str(e)}")
        raise

In [219]:
name="味全 每日C 100%纯橙汁 300ml_瓶 （新老包装随机）"
ele_df = load_excel("./饿了么-京东便利店（虹桥中心店）全量商品数据20251110.xlsx")
ele_df.head()

,商品ID,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,店内二级分类,...,想买人数,点赞数,最小订购数,标识,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid
0,950657120093,汤达人 日式豚骨拉面 83g(面饼55g+配料28g)/桶,NaN,6925303770563,3.49,8.3,4.3折,100,泡面速食卤蛋火腿,泡面拌面,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,950657120093
1,950103769331,康师傅 BIG大食桶老坛酸菜牛肉面 159g/桶,NaN,6920152424285,4.90,6.5,7.6折,200,泡面速食卤蛋火腿,康师傅面,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,950103769331
2,949473714726,上好佳 鲜虾片膨化食品 80克/袋,NaN,6926265313386,5.20,8.0,6.5折,100,膨化食品干吃脆面,薯条薯片,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,949473714726
3,949474714550,百威 9.7°P啤酒 500ml/听,NaN,6948960100078,0.01,9.5,0.1折,300,酒酒酒,精酿啤酒,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,949474714550
4,949480010428,统一茄皇 茄皇牛肉面 128g/桶,NaN,6925303796426,5.50,7.5,7.4折,75,泡面速食卤蛋火腿,康师傅面,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,949480010428


In [222]:
tokens = preprocess_candidate_tokens(ele_df)
find_top3_similar_combined(name,ele_df,tokens)


('949477802059-味全 每日C 100%纯橙汁 300ml/瓶 （新老包装随机）--1.0--1.0--1.0',
 '950658772111-味全 每日c葡萄汁饮料 300ml/瓶（新老包装随机）--0.75--0.6428571428571429--1.0',
 '949014663803-味全每日C 100%投入苹果汁 300ml/瓶--0.6499999999999999--0.5--1.0')

In [223]:
owner_df = load_excel("./美团-快驿点特价超市(虹桥店)全量商品信息20251109.xlsx")
owner_df.head()

,商品ID,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,店内二级分类,...,想买人数,点赞数,最小订购数,标识,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid
0,22639130381,【规格可选】30W充电套装 适用苹果8-14系列PD快充充电头 苹果手机数据线充电器 1个,单独 1米线,6978410500134,6.75,6.75,NaN,3,所搜商品,所搜商品,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,直插充电器,41430057383
1,22638543544,农夫山泉 饮用纯净水 550ml12瓶_包,550ml*12,6921168560424,9.90,22.00,NaN,34,推荐,推荐,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,包装饮用水,41429308607
2,22636514960,红牛Red Bull 维生素风味饮料 250ml_罐,250ml*1罐,6970640429988,8.07,8.23,NaN,7,推荐,推荐,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,运动功能饮料（非健字号）,41430928019
3,22640202093,百岁山 饮用天然矿泉水 570ml_瓶,NaN,06922255466476,2.99,2.99,NaN,6,推荐,推荐,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,天然矿泉水,41431324722
4,22740184269,康师傅茉莉蜜茶调味茶饮品 1000ml_瓶,1000ml*1瓶,6920459991985,5.54,5.65,NaN,1,推荐,推荐,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,茶饮料,41669396232


In [224]:
top1_list = []
top2_list = []
top3_list = []
output_path = "liyu附件1_补充Top3相似商品（ID-名称组合）.xlsx"

for idx, row in owner_df.iterrows():
    # 每处理100个商品显示进度（便于跟踪大量数据）
    if (idx + 1) % 100 == 0:
        print(f"已处理 {idx + 1}/{len(owner_df)} 个商品")

    product_name = row['商品名称']
    # 获取3个"ID-名称"组合（分别对应Top1、Top2、Top3）
    top1, top2, top3 = find_top3_similar_combined(product_name, ele_df, tokens)
    top1_list.append(top1)
    top2_list.append(top2)
    top3_list.append(top3)

owner_df['相似商品1（ID-名称）'] = top1_list  # 第三列：最相似
owner_df['相似商品2（ID-名称）'] = top2_list  # 第四列：次相似
owner_df['相似商品3（ID-名称）'] = top3_list  # 第五列：第三相似

# 5. 保存结果（不覆盖原文件）
owner_df.to_excel(output_path, index=False, engine='openpyxl')
print(f"\n处理完成！结果已保存到：{output_path}")
print("列说明：")
print(" - 第三列：相似商品1（ID-名称）→ 最相似商品")
print(" - 第四列：相似商品2（ID-名称）→ 次相似商品")
print(" - 第五列：相似商品3（ID-名称）→ 第三相似商品")
print("注：不足3个相似商品时，对应列为空字符串")

已处理 100/5405 个商品
已处理 200/5405 个商品
已处理 300/5405 个商品
已处理 400/5405 个商品
已处理 500/5405 个商品
已处理 600/5405 个商品
已处理 700/5405 个商品
已处理 800/5405 个商品
已处理 900/5405 个商品
已处理 1000/5405 个商品
已处理 1100/5405 个商品
已处理 1200/5405 个商品
已处理 1300/5405 个商品
已处理 1400/5405 个商品
已处理 1500/5405 个商品
已处理 1600/5405 个商品
已处理 1700/5405 个商品
已处理 1800/5405 个商品
已处理 1900/5405 个商品
已处理 2000/5405 个商品
已处理 2100/5405 个商品
已处理 2200/5405 个商品
已处理 2300/5405 个商品
已处理 2400/5405 个商品
已处理 2500/5405 个商品
已处理 2600/5405 个商品


KeyboardInterrupt: 